In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from Datathon.Utils.apache2Calc import *
from Datathon.Utils.getData import *
from Datathon.Utils.pipeFunctions import *

from sklearn.pipeline import Pipeline

from category_encoders import TargetEncoder , HelmertEncoder , JamesSteinEncoder,MEstimateEncoder
from category_encoders.woe import WOEEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from pyod.models.knn import KNN

from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import KernelPCA , TruncatedSVD

import numpy as np

from genetic_selection import GeneticSelectionCV

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0

In [2]:
adf = getTrainingData()
#adf = adf.dropna(thresh=adf.shape[0]*0.7 , axis=1)

cols_useless =[ "encounter_id", "hospital_id","patient_id" , "icu_id" , "gender" , "ethnicity"]
#adf = adf.drop(cols_useless,  axis=1)

#cat_cols = [c for c in getCategorialColumns(adf) if c not in cols_useless + ["hospital_death"]]
def tooManyMissing(adf):
    tdf = adf.dropna(thresh=adf.shape[0]*0.75 , axis=1)
    return list(set(adf.columns).difference(set(tdf.columns)))

tooManyMissing_cols = []#tooManyMissing(adf)

cat_cols = [c for c in getCategorialColumns(adf) if c not in ["hospital_death"] + tooManyMissing_cols]
num_cols = [c for c in getNumericColumns(adf) if c not in tooManyMissing_cols]
DEPENDENT_VARIABLE = getDependentVariable()

In [4]:
numeric_cols_pipe = Pipeline(steps=[
    ('mean impute' ,SimpleImputer(strategy="mean") )
    ,('Standard Scale' ,StandardScaler() )
])


cat_cols_pipe = Pipeline(steps=[
    ('Drop Useless' ,
        ColumnTransformer([
            ('Drop Useless Categorical Features' ,'drop' ,cols_useless )
        ],remainder='passthrough')
    )
    ,('Most Frequent impute' ,SimpleImputer(strategy="most_frequent"))
    ,('Encoding' , HelmertEncoder())
])

transform_pipe = Pipeline(steps=[
     ('Column Transform' ,
        ColumnTransformer([
            ('Drop Too Many Missing' , 'drop' , tooManyMissing_cols)
            ,('Cat Cols Pipe' ,cat_cols_pipe ,cat_cols )
            ,('Numeric Cols Pipe' ,numeric_cols_pipe ,num_cols )
        ])
    )
    #,('Drop Outlier Rows' ,RemoveOutliers())
    #,('Scale Everything' , StandardScaler())
    

])

In [5]:
X = adf.drop([DEPENDENT_VARIABLE] , axis=1)
y = adf[DEPENDENT_VARIABLE]
trX = transform_pipe.fit_transform(X,y)

In [6]:
from genetic_selection import GeneticSelectionCV
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression(solver="liblinear")


In [7]:
selector = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring="roc_auc",
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=5,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)

In [ ]:
selector.fit(trX , y)

Selecting features with genetic algorithm.
gen	nevals	avg                        	std                            	min                      	max                        
0  	50    	[  0.86114967 110.92      ]	[6.48235771e-03 7.15497030e+00]	[ 0.84485902 94.        ]	[  0.87244698 125.        ]
1  	30    	[  0.86549472 111.68      ]	[4.06013953e-03 6.35748378e+00]	[ 0.85575885 99.        ]	[  0.87479987 125.        ]
2  	34    	[  0.86877191 112.82      ]	[4.12549170e-03 6.17637434e+00]	[  0.85342936 101.        ]	[  0.87741427 125.        ]
3  	32    	[  0.87212298 114.2       ]	[2.46845863e-03 6.30555311e+00]	[  0.86694451 102.        ]	[  0.87737633 126.        ]
4  	28    	[  0.87371576 114.08      ]	[1.99195928e-03 6.17686004e+00]	[  0.86894582 101.        ]	[  0.87737633 126.        ]
5  	36    	[  0.87505449 113.98      ]	[1.59741059e-03 5.80168941e+00]	[  0.86989355 101.        ]	[  0.87754964 129.        ]
6  	28    	[  0.87608556 114.32      ]	[1.16213936e-03 5.33456652e+00]	[  

In [ ]:
selector.support_